In [135]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import pandas as pd

In [136]:
# Step 1: Set up the web driver (Make sure you have the correct version for your browser)

driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver')

<ipython-input-136-c5cbcde32321>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver')


In [137]:
# Step 2: Open the Georgetown directory page
# driver.get("https://contact.georgetown.edu")
driver.get("https://contact.georgetown.edu/advancedsearch/")

In [138]:
# Step 3: Select from the dropdown menu

dropdown = driver.find_element(By.NAME, "deptMatch")
select = Select(dropdown)
select.select_by_value("2") # select 'contains', while `<option value="2">contains</option>`

In [139]:
# Step 4: Input search term for law students

search_box = driver.find_element(By.NAME, "department")
search_box.send_keys("law center")
search_box.send_keys(Keys.RETURN)

In [140]:
# Step 5: Parse the page content with BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")

In [141]:
# get_email()

def get_netID_email(link):
    netID = link.split('/')[-1]
    email = netID + '@georgetown.edu'
    return (netID, email)

In [142]:
# Key step: Extract relevant fields
names = []
phones = []
affiliations = []
statuses = []
emails = []
links = []

persons = soup.find_all("td")

# Adjust the selectors based on the directory's structure
for i in range(len(persons)):
    if i % 3 == 0:
        name = persons[i].find("a").text if persons[i].find("a") else None
        link = persons[i].find("a")['href'] if persons[i].find("a")['href'] else None
        email = get_netID_email(link)[1]
        
        names.append(name)
        links.append(link)
        emails.append(email)
    elif i % 3 == 1:
        status = persons[i].contents[0].strip() if persons[i].contents[0] else None
        affiliation = persons[i].find("small").text if persons[i].find("small") else None
        statuses.append(status)
        affiliations.append(affiliation)
    else:
        pass

In [148]:
# Step 7: Store the data in a pandas DataFrame and export to CSV
data = {
    # "Name": names,
    "Status": statuses,
    "Affiliation": affiliations,
    "Email": emails,
    # "Profile Link": links
}
df = pd.DataFrame(data)

In [149]:
# Step 8: Save to CSV
df.to_csv("georgetown_law_center_directory.csv", index=False)
df['Email'].to_csv("emails.csv", index=False)

In [145]:
# Close the browser
driver.quit()